# Cleanup and Exploration of 2022 AQ Data (Part 1)

Jupyter Notebook split to reduce cumulative memory burden. 

In [1]:
import pandas as pd
import json as json

In [2]:
df = pd.read_csv('2022 AQ.csv')

In [3]:
df.head()

,date,parameter,location,value,unit,city,attribution,averagingperiod,coordinates,country,sourcename,sourcetype,mobile
0,"{utc=2022-04-24T03:00:00.000Z, local=2022-04-2...",o3,Huntsville Old Airpo,0.04,ppm,Huntsville,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.6917, longitude=-86.5917}",US,AirNow,government,False
1,"{utc=2022-04-24T03:00:00.000Z, local=2022-04-2...",pm25,Huntsville Old Airpo,10.50,µg/m³,Huntsville,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.6917, longitude=-86.5917}",US,AirNow,government,False
2,"{utc=2022-04-24T03:00:00.000Z, local=2022-04-2...",pm10,Ft McD Yuma Frank-AZ,13.00,µg/m³,Phoenix-Mesa-Scottsdale,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.6292, longitude=-111.6769}",US,AirNow,government,False
3,"{utc=2022-04-24T03:00:00.000Z, local=2022-04-2...",pm10,Brawley - 220 Main S,26.00,µg/m³,El Centro,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=32.9783, longitude=-115.5383}",US,AirNow,government,False
4,"{utc=2022-04-24T03:00:00.000Z, local=2022-04-2...",pm25,Brawley - 220 Main S,4.80,µg/m³,El Centro,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=32.9783, longitude=-115.5383}",US,AirNow,government,False


The entry is not valid json.  The field will need processing code.

In [4]:
def validate_json_fields(field: str) -> bool:
    """This function validates a JSON entry is valid.  Returns True if so, False if otherwise."""
    try:
        json.loads(field)
        return True
    except ValueError:
        return False

In [5]:
def validate_json_column(series: pd.Series):
    temp_df = pd.DataFrame()
    temp_df['values'] = series
    temp_df['validity'] = series.apply(validate_json_fields)
    return temp_df

## Validation of composite fields as json

In [6]:
output = validate_json_column(df['date'])

In [7]:
output.head()

,values,validity
0,"{utc=2022-04-24T03:00:00.000Z, local=2022-04-2...",False
1,"{utc=2022-04-24T03:00:00.000Z, local=2022-04-2...",False
2,"{utc=2022-04-24T03:00:00.000Z, local=2022-04-2...",False
3,"{utc=2022-04-24T03:00:00.000Z, local=2022-04-2...",False
4,"{utc=2022-04-24T03:00:00.000Z, local=2022-04-2...",False


In [8]:
output[output['validity'] == True].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


In [9]:
output = validate_json_column(df['attribution'])
output.head()

,values,validity
0,"[{name=US EPA AirNow, url=http://www.airnow.go...",False
1,"[{name=US EPA AirNow, url=http://www.airnow.go...",False
2,"[{name=US EPA AirNow, url=http://www.airnow.go...",False
3,"[{name=US EPA AirNow, url=http://www.airnow.go...",False
4,"[{name=US EPA AirNow, url=http://www.airnow.go...",False


In [10]:
output[output['validity'] == True].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


In [11]:
output = validate_json_column(df['averagingperiod'])
output.head()

,values,validity
0,"{unit=hours, value=1.0}",False
1,"{unit=hours, value=1.0}",False
2,"{unit=hours, value=1.0}",False
3,"{unit=hours, value=1.0}",False
4,"{unit=hours, value=1.0}",False


In [12]:
output[output['validity'] == True].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


In [13]:
output = validate_json_column(df['coordinates'])
output.head()

,values,validity
0,"{latitude=34.6917, longitude=-86.5917}",False
1,"{latitude=34.6917, longitude=-86.5917}",False
2,"{latitude=33.6292, longitude=-111.6769}",False
3,"{latitude=32.9783, longitude=-115.5383}",False
4,"{latitude=32.9783, longitude=-115.5383}",False


In [14]:
output[output['validity'] == True].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


All of the composite fields are in a "JSON-like format"  need to convert to JSON.

## Reprocessing the Multivalue Fields into Valid JSON

In [15]:
def correct_field_to_json(arg: str) -> str:
    string = arg.lstrip("{{")
    string = string.rstrip("]}")
    strings = string.split(",")
    new_strings = []
    for entry in strings:
        index = entry.find('=')
        entry = '"' + entry[0:index] + '"' + ':' + '"'+ entry[index+1:len(entry)]+'"'
        new_strings.append(entry)
    output_string = (',').join(entry for entry in new_strings)
    output_string = '{' + output_string + '}'
    return output_string

In [16]:
def correct_json_column(series: pd.Series, column_name: str) -> pd.DataFrame:
    temp_df = pd.DataFrame()
    temp_df[column_name] = series.apply(correct_field_to_json)
    return temp_df

In [17]:
test = correct_json_column(df['date'], 'test date conversion')
results = validate_json_column(test['test date conversion'])
results[results['validity'] == False].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


In [18]:
df['cleaneddate'] = test

In [19]:
test = correct_json_column(df['attribution'], 'test attribute conversion')
results = validate_json_column(test['test attribute conversion'])
results[results['validity'] == False].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


In [20]:
results.head()

,values,validity
0,"{""[{name"":""US EPA AirNow"","" url"":""http://www.a...",True
1,"{""[{name"":""US EPA AirNow"","" url"":""http://www.a...",True
2,"{""[{name"":""US EPA AirNow"","" url"":""http://www.a...",True
3,"{""[{name"":""US EPA AirNow"","" url"":""http://www.a...",True
4,"{""[{name"":""US EPA AirNow"","" url"":""http://www.a...",True


In [21]:
df['cleanedattribution'] = test

In [22]:
test = correct_json_column(df['averagingperiod'], 'test averaging period conversion')
results = validate_json_column(test['test averaging period conversion'])
results[results['validity'] == False].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


In [23]:
df['cleanedaveragingperiod'] = test

In [24]:
test = correct_json_column(df['coordinates'], 'test coordinate conversion')
results = validate_json_column(test['test coordinate conversion'])
results[results['validity'] == False].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


In [25]:
df['cleanedcoordinates'] = test

In [26]:
df.head()

,date,parameter,location,value,unit,city,attribution,averagingperiod,coordinates,country,sourcename,sourcetype,mobile,cleaneddate,cleanedattribution,cleanedaveragingperiod,cleanedcoordinates
0,"{utc=2022-04-24T03:00:00.000Z, local=2022-04-2...",o3,Huntsville Old Airpo,0.04,ppm,Huntsville,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.6917, longitude=-86.5917}",US,AirNow,government,False,"{""utc"":""2022-04-24T03:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""34.6917"","" longitude"":""-86.5917""}"
1,"{utc=2022-04-24T03:00:00.000Z, local=2022-04-2...",pm25,Huntsville Old Airpo,10.50,µg/m³,Huntsville,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.6917, longitude=-86.5917}",US,AirNow,government,False,"{""utc"":""2022-04-24T03:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""34.6917"","" longitude"":""-86.5917""}"
2,"{utc=2022-04-24T03:00:00.000Z, local=2022-04-2...",pm10,Ft McD Yuma Frank-AZ,13.00,µg/m³,Phoenix-Mesa-Scottsdale,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.6292, longitude=-111.6769}",US,AirNow,government,False,"{""utc"":""2022-04-24T03:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.6292"","" longitude"":""-111.6769""}"
3,"{utc=2022-04-24T03:00:00.000Z, local=2022-04-2...",pm10,Brawley - 220 Main S,26.00,µg/m³,El Centro,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=32.9783, longitude=-115.5383}",US,AirNow,government,False,"{""utc"":""2022-04-24T03:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""32.9783"","" longitude"":""-115.5383""}"
4,"{utc=2022-04-24T03:00:00.000Z, local=2022-04-2...",pm25,Brawley - 220 Main S,4.80,µg/m³,El Centro,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=32.9783, longitude=-115.5383}",US,AirNow,government,False,"{""utc"":""2022-04-24T03:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""32.9783"","" longitude"":""-115.5383""}"


In [27]:
import ujson as ujson

In [28]:
df.to_csv('2022 AQ Clean.csv', index=False)

##### 